Ha: Work done so far: change all data types, drop some corrupted rows with at start_time and end_time
Suggestion for handling null values at start_station_id: we may want to make use of latitude and longtitude? notice below when 
i sort the dataframe by bike_id, it shows the route that the one bike commute. So maybe find out which row has the same latitude/longtitude which the missing value row and 
and insert the id accordingly.


In [7]:
# import all relevant libraries
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import seaborn as sns
sns.set()
sns.set_style("white")
sns.set_palette("GnBu_d")

# also import these "new" libraries 
# Note: you may have to download an add them to your environment (using e.g. 'conda install -c conda-forge folium')
import folium
from folium import plugins
from folium.plugins import HeatMap
from datetime import datetime #for working with times objects
from datetime import timedelta #for working with times objects
import math
import random

import data 

In [8]:
#load csv file and et the index column by 0
bikeRen = pd.read_csv("baywheels_2019", encoding = "ISO-8859-1", index_col=0) 
bikeRen.head(20)

C:\Users\riven\AppData\Local\Temp/ipykernel_23132/4179206828.py:2: DtypeWarning: Columns (3,4,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  bikeRen = pd.read_csv("baywheels_2019", encoding = "ISO-8859-1", index_col=0)


,start_time,end_time,start_station_id,start_station_lat,start_station_lon,end_station_id,end_station_lat,end_station_lon,bike_id
0,2019-10-28 09:46:47,2019-10-28 09:51:05,50.0,37.780526,-122.390288,453.0,37.777934,-122.396973,12424.0
1,2019-08-10 19:31:02,2019-08-10 19:35:05,285.0,37.783521,-122.431158,74.0,37.776435,-122.426244,1718.0
2,2019-05-27 13:18:58,2019-05-27 13:46:37,10.0,37.795393,-122.40477,399.0,37.802636,-122.436289,608.0
3,2019-07-24 22:47:58,2019-07-24 22:52:48,106.0,37.763242,-122.430675,72.0,37.772406,-122.43565,551642.0
4,2019-04-28 17:26:22,2019-04-28 17:31:54,80.0,37.775235,-122.397437,50.0,37.780526,-122.390288,1103.0
5,2019-04-17 19:39:05,2019-04-17 19:49:26,100.0,37.7671,-122.410662,74.0,37.776435,-122.426244,1473.0
6,2019-02-26 08:11:01,2019-02-26 08:22:58,67.0,37.776639,-122.395526,8.0,37.799953,-122.398525,4984.0
7,2019-02-02 09:23:15,2019-02-02 09:34:20,44.0,37.781074,-122.411738,55.0,37.777053,-122.429558,2666.0
8,2019-05-07 12:08:01,2019-05-07 12:12:00,9.0,37.798572,-122.400869,6.0,37.80477,-122.403234,1961.0
9,2019-09-04 21:58:19,2019-09-04 22:09:07,86.0,37.769305,-122.426826,39.0,37.778999,-122.436861,9717.0


In [9]:
print(bikeRen.info())
print(bikeRen.describe())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2507003 entries, 0 to 2507002
Data columns (total 9 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   start_time         object
 1   end_time           object
 2   start_station_id   object
 3   start_station_lat  object
 4   start_station_lon  object
 5   end_station_id     object
 6   end_station_lat    object
 7   end_station_lon    object
 8   bike_id            object
dtypes: object(9)
memory usage: 191.3+ MB
None
       start_time end_time  start_station_id  start_station_lat  \
count     2507003  2507003         2426269.0       2.507003e+06   
unique    2304544  2303004             867.0       7.712800e+04   
top       Failure  Failure              58.0       3.777662e+01   
freq           10       10           34467.0       3.446700e+04   

        start_station_lon  end_station_id  end_station_lat  end_station_lon  \
count        2.507003e+06       2424101.0     2.507003e+06     2.507003e+06   
unique

In [10]:
bikeRen.isna().sum()

start_time               0
end_time                 0
start_station_id     80734
start_station_lat        0
start_station_lon        0
end_station_id       82902
end_station_lat          0
end_station_lon          0
bike_id                  0
dtype: int64

In [11]:
bikeRen["bike_id"]= pd.to_numeric(bikeRen["bike_id"], errors='coerce')

print(bikeRen.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2507003 entries, 0 to 2507002
Data columns (total 9 columns):
 #   Column             Dtype  
---  ------             -----  
 0   start_time         object 
 1   end_time           object 
 2   start_station_id   object 
 3   start_station_lat  object 
 4   start_station_lon  object 
 5   end_station_id     object 
 6   end_station_lat    object 
 7   end_station_lon    object 
 8   bike_id            float64
dtypes: float64(1), object(8)
memory usage: 191.3+ MB
None


In [12]:
bikeRen["bike_id"]= bikeRen["bike_id"].astype('Int64')
print(bikeRen.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2507003 entries, 0 to 2507002
Data columns (total 9 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   start_time         object
 1   end_time           object
 2   start_station_id   object
 3   start_station_lat  object
 4   start_station_lon  object
 5   end_station_id     object
 6   end_station_lat    object
 7   end_station_lon    object
 8   bike_id            Int64 
dtypes: Int64(1), object(8)
memory usage: 193.7+ MB
None


In [13]:
bikeRen["start_station_lat"]= pd.to_numeric(bikeRen["start_station_lat"], errors='coerce')
bikeRen.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2507003 entries, 0 to 2507002
Data columns (total 9 columns):
 #   Column             Dtype  
---  ------             -----  
 0   start_time         object 
 1   end_time           object 
 2   start_station_id   object 
 3   start_station_lat  float64
 4   start_station_lon  object 
 5   end_station_id     object 
 6   end_station_lat    object 
 7   end_station_lon    object 
 8   bike_id            Int64  
dtypes: Int64(1), float64(1), object(7)
memory usage: 193.7+ MB


In [14]:
bikeRen["start_station_lon"]=pd.to_numeric(bikeRen["start_station_lon"], errors='coerce')
bikeRen["end_station_lat"]=pd.to_numeric(bikeRen["end_station_lat"], errors='coerce')
bikeRen["end_station_lon"]=pd.to_numeric(bikeRen["end_station_lon"], errors='coerce')
bikeRen.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2507003 entries, 0 to 2507002
Data columns (total 9 columns):
 #   Column             Dtype  
---  ------             -----  
 0   start_time         object 
 1   end_time           object 
 2   start_station_id   object 
 3   start_station_lat  float64
 4   start_station_lon  float64
 5   end_station_id     object 
 6   end_station_lat    float64
 7   end_station_lon    float64
 8   bike_id            Int64  
dtypes: Int64(1), float64(4), object(4)
memory usage: 193.7+ MB


In [15]:
bikeRen["start_station_id"] =pd.to_numeric(bikeRen["start_station_id"], errors='coerce')
bikeRen["start_station_id"]= bikeRen["start_station_id"].astype("Int64")
bikeRen["end_station_id"] =pd.to_numeric(bikeRen["end_station_id"], errors='coerce')
bikeRen["end_station_id"]= bikeRen["end_station_id"].astype("Int64")

In [16]:
bikeRen.isnull().sum()

start_time               0
end_time                 0
start_station_id     80744
start_station_lat       10
start_station_lon       10
end_station_id       82912
end_station_lat         10
end_station_lon         10
bike_id                 11
dtype: int64

In [17]:
#create a new data frame that containts all failure rows from bikeRent
rent_failure = bikeRen["start_time"].str.contains('Failure')

data_failure = bikeRen[rent_failure]
data_failure



,start_time,end_time,start_station_id,start_station_lat,start_station_lon,end_station_id,end_station_lat,end_station_lon,bike_id
381556,Failure,Failure,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>
485428,Failure,Failure,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>
621119,Failure,Failure,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>
713198,Failure,Failure,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>
1150787,Failure,Failure,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>
1558857,Failure,Failure,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>
1676023,Failure,Failure,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>
1697405,Failure,Failure,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>
1999499,Failure,Failure,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>
2249494,Failure,Failure,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>


drop all rows in data failure  


In [18]:
#remove all rows that match failure rows
bikeRen = bikeRen.drop(data_failure.index, axis= 0)
display(data_failure)


,start_time,end_time,start_station_id,start_station_lat,start_station_lon,end_station_id,end_station_lat,end_station_lon,bike_id
381556,Failure,Failure,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>
485428,Failure,Failure,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>
621119,Failure,Failure,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>
713198,Failure,Failure,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>
1150787,Failure,Failure,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>
1558857,Failure,Failure,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>
1676023,Failure,Failure,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>
1697405,Failure,Failure,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>
1999499,Failure,Failure,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>
2249494,Failure,Failure,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>


In [19]:
check_failure = bikeRen["start_time"].str.contains('Failure')
#a method to checks whether there is still failure row
def is_clear_of_failure(s) :
    clear = True
    for b in s:
        if b == True:
            clear = False
    print(clear)
    
is_clear_of_failure(check_failure)

True


In [20]:
bikeRen["start_time"]= pd.to_datetime(bikeRen["start_time"],format ='%Y-%m-%d %H:%M:%S')

In [21]:
bikeRen["end_time"]= pd.to_datetime(bikeRen["end_time"],format ='%Y-%m-%d %H:%M:%S')

In [22]:
bikeRen.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2506993 entries, 0 to 2507002
Data columns (total 9 columns):
 #   Column             Dtype         
---  ------             -----         
 0   start_time         datetime64[ns]
 1   end_time           datetime64[ns]
 2   start_station_id   Int64         
 3   start_station_lat  float64       
 4   start_station_lon  float64       
 5   end_station_id     Int64         
 6   end_station_lat    float64       
 7   end_station_lon    float64       
 8   bike_id            Int64         
dtypes: Int64(3), datetime64[ns](2), float64(4)
memory usage: 198.4 MB


In [23]:
bikeRen.head(10)

,start_time,end_time,start_station_id,start_station_lat,start_station_lon,end_station_id,end_station_lat,end_station_lon,bike_id
0,2019-10-28 09:46:47,2019-10-28 09:51:05,50,37.780526,-122.390288,453,37.777934,-122.396973,12424
1,2019-08-10 19:31:02,2019-08-10 19:35:05,285,37.783521,-122.431158,74,37.776435,-122.426244,1718
2,2019-05-27 13:18:58,2019-05-27 13:46:37,10,37.795393,-122.404770,399,37.802636,-122.436289,608
3,2019-07-24 22:47:58,2019-07-24 22:52:48,106,37.763242,-122.430675,72,37.772406,-122.435650,551642
4,2019-04-28 17:26:22,2019-04-28 17:31:54,80,37.775235,-122.397437,50,37.780526,-122.390288,1103
5,2019-04-17 19:39:05,2019-04-17 19:49:26,100,37.767100,-122.410662,74,37.776435,-122.426244,1473
6,2019-02-26 08:11:01,2019-02-26 08:22:58,67,37.776639,-122.395526,8,37.799953,-122.398525,4984
7,2019-02-02 09:23:15,2019-02-02 09:34:20,44,37.781074,-122.411738,55,37.777053,-122.429558,2666
8,2019-05-07 12:08:01,2019-05-07 12:12:00,9,37.798572,-122.400869,6,37.804770,-122.403234,1961
9,2019-09-04 21:58:19,2019-09-04 22:09:07,86,37.769305,-122.426826,39,37.778999,-122.436861,9717


In [24]:
bikeRen = bikeRen.sort_values(by='bike_id')
bikeRen.head(10)
bikeRen[bikeRen["start_station_id"].isnull()]

,start_time,end_time,start_station_id,start_station_lat,start_station_lon,end_station_id,end_station_lat,end_station_lon,bike_id
1956039,2019-02-23 17:57:52,2019-02-23 18:26:20,<NA>,37.410000,-121.950000,<NA>,37.410000,-121.950000,3975
2298364,2019-02-27 14:06:51,2019-02-27 14:13:31,<NA>,37.410000,-121.950000,<NA>,37.410000,-121.960000,3975
193315,2019-03-11 20:46:18,2019-03-11 20:48:55,<NA>,37.420000,-121.940000,<NA>,37.420000,-121.940000,3975
826239,2019-03-11 12:05:53,2019-03-11 12:09:38,<NA>,37.420000,-121.940000,<NA>,37.420000,-121.940000,3975
1321395,2019-03-12 08:44:08,2019-03-12 08:49:39,<NA>,37.420000,-121.940000,<NA>,37.410000,-121.950000,3975
...,...,...,...,...,...,...,...,...,...
813813,2019-12-08 01:15:35,2019-12-08 01:16:41,<NA>,37.775925,-122.392526,<NA>,37.776050,-122.392549,999941
1425376,2019-12-03 10:24:02,2019-12-03 10:25:51,<NA>,37.775769,-122.392549,<NA>,37.775718,-122.392675,999941
1434984,2019-12-27 12:51:19,2019-12-27 12:52:25,<NA>,37.775912,-122.392578,<NA>,37.775857,-122.392584,999941
1274746,2019-12-03 11:05:06,2019-12-03 11:07:02,<NA>,37.775776,-122.392644,<NA>,37.775815,-122.392665,999941


In [94]:
#new dataframe that can be used to merge the missing values
df_station_info_pre = bikeRen[['start_station_id', 'start_station_lat', 'start_station_lon']]
df_station_info=df_station_info_pre.dropna().copy()
df_station_info.drop_duplicates(inplace=True)


,start_station_id,start_station_lat,start_station_lon
820394,436,37.870310,-122.409794
315417,92,37.773503,-122.402113
1052707,195,37.791826,-122.410040
1590064,395,37.778562,-122.415787
1971590,231,37.352950,-122.287678
...,...,...,...
2382399,262,37.869785,-122.287936
207062,65,37.830296,-122.289430
879163,329,37.344018,-121.915063
1571879,428,37.774661,-122.416308


In [54]:
df_station_info.sort_values(by= 'start_station_id')


,start_station_id,start_station_lat,start_station_lon
2456862,3,37.786375,-122.404904
2065268,4,37.785881,-122.408915
377857,5,37.783899,-122.408445
209926,6,37.804770,-122.403234
1584959,7,37.804562,-122.271738
...,...,...,...
1041035,492,37.790803,-122.432590
1839715,493,37.776664,-122.407908
231213,494,37.326282,-121.904911
1707561,495,37.343208,-121.884390


In [92]:
from sklearn.cluster import KMeans
bikeRen.start_station_id.unique()


<IntegerArray>
[ 436,   92,  195,  395,  231,  408,  131,  380,  228,  345,
 ...
  494,  444,  498,  484,  429,  329,   65,  428,  449, <NA>]
Length: 437, dtype: Int64

In [95]:
kmeans = KMeans(n_clusters=436)



In [98]:
# Fit the model to the station coordinates
X = df_station_info[['start_station_lat', 'start_station_lon']].values



kmeans.fit(X)

# Predict the cluster labels for the missing coordinates
# missing_coords = bikeRen[bikeRen.start_station_id.isna()].copy().loc[:,'start_station_lat':'start_station_lon']
# missing_labels = kmeans.predict(missing_coords)

# missing_data = bikeRen[bikeRen.start_station_id.isna()]

# Assign the closest station id to each missing coordinate

# for i, label in enumerate(missing_labels):
#     closest_station = df_station_info[kmeans.labels_ == label].iloc[-1]['start_station_id']
#     print(f"Missing station id {i+1} is closest to {closest_station}.")
    
    
for i, row in bikeRen[bikeRen.start_station_id.isna()].iterrows():
    
        coord = [[row['start_station_lat'],row['start_station_lon']]]
        prediction = kmeans.predict(coord)
        closest_station = df_station_info[kmeans.labels_ == prediction].iloc[-1]['start_station_id']
        print(f"Missing station id {i+1} is closest to {closest_station}.")
        
        bikeRen.at[i, 'start_station_id'] = closest_station
        
        

    

# test = bikeRen[bikeRen['start_station_id'].isna()].apply(fill_station_id, axis = 1)
# test





c:\Users\riven\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Missing station id 1751892 is closest to 114.0.
Missing station id 1237261 is closest to 91.0.
Missing station id 1272998 is closest to 78.0.
Missing station id 2126893 is closest to 63.0.
Missing station id 164625 is closest to 321.0.
Missing station id 1914206 is closest to 44.0.
Missing station id 1094014 is closest to 458.0.
Missing station id 1600193 is closest to 130.0.
Missing station id 1619966 is closest to 323.0.
Missing station id 1948832 is closest to 104.0.
Missing station id 23414 is closest to 386.0.
Missing station id 2488413 is closest to 386.0.
Missing station id 1676426 is closest to 136.0.
Missing station id 858147 is closest to 386.0.
Missing station id 2289325 is closest to 25.0.
Missing station id 463979 is closest to 324.0.
Missing station id 936562 is closest to 104.0.
Missing station id 1187372 is closest to 364.0.
Missing station id 2071244 is closest to 75.0.
Missing station id 231230 is closest to 323.0.
Missing station id 1166625 is closest to 139.0.
Missi

KeyboardInterrupt: 

In [99]:
bikeRen.isna().sum()

start_time               0
end_time                 0
start_station_id     54564
start_station_lat        0
start_station_lon        0
end_station_id       82902
end_station_lat          0
end_station_lon          0
bike_id                  1
dtype: int64

In [39]:
closest_station.dtype

dtype('float64')

In [42]:
closest_station = closest_station.astype('int64')
closest_station

81